<a href="https://colab.research.google.com/github/prajeet26/Twitter_disaster_classification_using_BERT/blob/main/Disaster_tweet_classi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTING LIBS

In [4]:
#lets import tqdm for progress bar 
! pip install tqdm
from tqdm.notebook import tqdm 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import string
import re

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics import f1_score, accuracy_score

import random
import warnings
import time
import datetime

from wordcloud import WordCloud, STOPWORDS
from collections import Counter, defaultdict
from PIL import Image
import spacy
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.5/en_core_web_sm-2.2.5.tar.gz
import en_core_web_sm

from matplotlib.ticker import MaxNLocator
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches

#to maintain consistent results
stop = set(stopwords.words('english'))
plt.style.use('fivethirtyeight')
sns.set(font_scale=1.5)
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250
warnings.filterwarnings('ignore')

! pip install transformers

import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 292 kB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-py3-none-any.whl size=12011737 sha256=b94827915f2c747eb9c502f36fd0bfac9327d87e89333a2e3df2a05cf767e4b7
  Stored in directory: /root/.cache/pip/wheels/51/19/da/a3885266a3c241aff0ad2eb674ae058fd34a4870fef1c0a5a0
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.3.0
    Uninstalling en-core-web-sm-3.3.0:
      Successfully uninstalled en-core-web-sm-3.3.0


/usr/local/lib/python3.7/dist-packages/spacy/util.py:845: UserWarning: [W094] Model 'en_core_web_sm' (2.2.5) specifies an under-constrained spaCy version requirement: >=2.2.2. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.3.1,<3.4.0
  warnings.warn(warn_msg)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.8 MB/s 
     |████████████████████████████████| 101 kB 13.3 MB/s 
     |████████████████████████████████| 6.6 MB 29.1 MB/s 
     |████████████████████████████████| 596 kB 48.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


MOUNTING DRIVE

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
trainvz = pd.read_csv('/content/drive/MyDrive/disastertrain.csv')
testvz = pd.read_csv('/content/drive/MyDrive/disastertest.csv')

In [10]:
trainvz.sample(100)


,id,keyword,location,text,target
4964,7075,meltdown,NOLA ?? TX,?? ice age: the meltdown,0
3984,5661,floods,South of D.C.,Slip Sliding Away - Flash Floods Info for Writ...,1
6242,8914,snowstorm,London,Can you list 5 reasons why a London #TubeStrik...,0
1178,1698,bridge%20collapse,"Leeds, England",Listening to Blowers and Tuffers on the Aussie...,0
3004,4317,dust%20storm,D(M)V,@RetiredFilth people in sydney woke up to the ...,1
4346,6171,hijack,Nigeria,Tension In Bayelsa As Patience Jonathan Plans ...,1
63,91,ablaze,"Concord, CA",@Navista7 Steve these fires out here are somet...,1
541,787,avalanche,Philippines,You are the avalanche. One world away. My make...,0
7325,10485,wildfire,"Tucson, AZ",Does the #FingerRockFire make you wonder 'am I...,1
360,518,army,NaN,One Direction Is my pick for http://t.co/q2eBl...,0


In [11]:
testvz.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [13]:
print(trainvz.shape)
print(testvz.shape)

(7613, 5)
(3263, 4)


*cleaning data*

In [23]:
def removeURL(text):
  text=re.sub(r"https?://\S+|www\.\S+","",text)
  return text

def removeEMG(text):
  emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
  return emoji_pattern.sub(r'', text)

def removeHTML(text):
  text=re.sub(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});","",text)
  return text

def removemen_hash(text):
    text = re.sub(r"@\S+","",text)
    text = re.sub(r"#","",text)
    return text

def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

#NOW APPLYING FUNCTIONS TO THE TEXT TO CLEAN

trainvz['text_clean'] = trainvz['text'].apply(lambda x: removeURL(x))
trainvz['text_clean'] = trainvz['text_clean'].apply(lambda x: removeEMG(x))
trainvz['text_clean'] = trainvz['text_clean'].apply(lambda x: removeHTML(x))
trainvz['text_clean'] = trainvz['text_clean'].apply(lambda x: removemen_hash(x))
trainvz['text_clean'] = trainvz['text_clean'].apply(lambda x: remove_punct(x))
  
trainvz['tokenized'] = trainvz['text_clean'].apply(word_tokenize)                               #tokenize the text into words
trainvz['lower']=trainvz['tokenized'].apply(lambda x:[word.lower() for word in x])              #lower case all
trainvz['stop_remo']=trainvz['lower'].apply(lambda x:[word for word in x if word not in stop])  #remove stopwords
trainvz['postags']=trainvz['stop_remo'].apply(nltk.pos_tag)                                     #tagging each word
trainvz.head()



,id,keyword,location,text,target,text_clean,tokenized,lower,upper,stop_remo,postags
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this earthquake Ma...,"[Our, Deeds, are, the, Reason, of, this, earth...","[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...","[deeds, reason, earthquake, may, allah, forgiv...","[(deeds, NNS), (reason, NN), (earthquake, NN),..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask Canada,"[Forest, fire, near, La, Ronge, Sask, Canada]","[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]","[(forest, JJS), (fire, NN), (near, IN), (la, J..."
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to shelter in place are be...,"[All, residents, asked, to, shelter, in, place...","[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o...","[residents, asked, shelter, place, notified, o...","[(residents, NNS), (asked, VBD), (shelter, JJ)..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,"[13000, people, receive, wildfires, evacuation...","[13000, people, receive, wildfires, evacuation...","[13000, people, receive, wildfires, evacuation...","[13000, people, receive, wildfires, evacuation...","[(13000, CD), (people, NNS), (receive, JJ), (w..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby Alaska as s...,"[Just, got, sent, this, photo, from, Ruby, Ala...","[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi...","[got, sent, photo, ruby, alaska, smoke, wildfi...","[(got, VBD), (sent, JJ), (photo, NN), (ruby, N..."
